<a href="https://www.kaggle.com/code/riturajbaruah/mvp-seller-data-for-queried-city-product?scriptVersionId=283202005" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [ ]:
from google.genai import types
from google.adk.agents import Agent
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components have been imported successfully.")

In [74]:
# Helper Functions
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [6]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [7]:
from typing import List, Dict, TypedDict

class SellerRow(TypedDict):
    """
    Represents one seller row in the table.

    This is for type clarity; ADK does not require it,
    but it helps us keep a consistent schema.
    """
    seller_entity_name: str
    address: str
    phone_number: str
    annual_turnover_cr: float
    gst: str
    google_rating: float
    rating_count: int
    matched_product: str
    city: str
print("✅ Seller table structure defined")

✅ Seller table structure defined


In [52]:
# Summarizer Agent: Its job is to create a summary the information that it receives in JSON format or some other structrure.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a summary - a 2-3 liner based on the research findings""",
    # ignore this part- output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


In [75]:
#SerpAPI tool to fetch seller entity data (GST, Annual turnover, Phone Number, Address etc. of a seller who sells {var} in city var}

import requests

def serpapi_google_maps_seller_search(city: str, product: str) -> dict:
    """
    Fetch sellers for a product in a specific city using SerpAPI Google Maps engine.

    Args:
        city (str): City to search in
        product (str): Product keyword

    Returns:
        dict: {
            "status": "success",
            "sellers": [ ...rows... ]
        }
        OR
        {
            "status": "error",
            "error_message": "Something went wrong"
        }
    """

    SERP_API_KEY = "bfc82ad31a9962303f4b8068637b2289057410e399713611646ee2cb961b9d0c"   # the key

    url = "https://serpapi.com/search"
    params = {
        "engine": "google_maps",
        "q": f"{product} in {city}",
        "api_key": SERP_API_KEY
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()
    except Exception as e:
        return {"status": "error", "error_message": str(e)}

    if "local_results" not in data:
        return {"status": "error", "error_message": "No results from SerpAPI"}

    sellers = []

    for item in data.get("local_results", []):
        sellers.append({
            "seller_entity_name": item.get("title"),
            "address": item.get("address"),
            "phone_number": item.get("phone"),
            "annual_turnover_cr": None,       # can be enriched later
            "gst": None,                      # can be enriched later
            "google_rating": item.get("rating"),
            "rating_count": item.get("reviews"),
            "matched_product": product,
            "city": city,
        })

    return {
        "status": "success",
        "sellers": sellers
    }

print("✅ SerpAPI tool function created")


✅ SerpAPI tool function created


In [76]:
SerpAPI_Seller_Agent = LlmAgent(
    name="SerpAPI_Seller_Agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
You are a Seller Discovery Agent.

When the user provides a 'city' and 'product':

1. Use `serpapi_google_maps_seller_search(city, product)` to get seller data.
2. ALWAYS check the `status` field.
3. If status = "success":
       return a clean structured table-like JSON containing:
       seller_entity_name, address, phone_number,
       annual_turnover_cr, gst, google_rating,
       rating_count, matched_product, city.
4. If status = "error":
       explain the error clearly to the user.
""",
    tools=[serpapi_google_maps_seller_search],
    output_key="research_findings",
)

print("✅ SerpAPI Seller Agent created with functional tools (ADK compliant)")
print(">> Available tools:")
print("  • serpapi_google_maps_seller_search - Fetches seller data from SerpAPI Google Maps")


✅ SerpAPI Seller Agent created with functional tools (ADK compliant)
>> Available tools:
  • serpapi_google_maps_seller_search - Fetches seller data from SerpAPI Google Maps


Let's follow the same best practices to define our second tool `get_exchange_rate`.

In [55]:
seller_runner = InMemoryRunner(agent=root_agent)
print("🚀 Principal Root Agent created")

🚀 Principal Root Agent created


In [61]:

# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="root_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a master coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `SerpAPI_Seller_Agent` tool to get -
    a clean structured table-like JSON containing:
       seller_entity_name, address, phone_number,
       annual_turnover_cr, gst, google_rating,
       rating_count, matched_product, city.
       
2. Next, after receiving the research findings, you MUST call the `summarizer_agent` tool to provide additional information based on the relevant information

3. Finally, present the final results clearly to the user as your response - publish the final results from both Step 1 and Step 2 - don't exclude any detail from Step 1.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(SerpAPI_Seller_Agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")



✅ root_agent created.


In [66]:
response = await seller_runner.run_debug(
    "City = Mumbai, Product = TMT Bars. Fetch seller list and also, generate a summary."
)



 ### Continue session: debug_session_id

User > City = Mumbai, Product = TMT Bars. Fetch seller list and also, generate a summary.
root_agent > The following sellers offer TMT Bars in Mumbai:

| seller_entity_name          | address                                                                                                       | phone_number      | annual_turnover_cr   | gst   | google_rating   | rating_count   | matched_product   | city   |
| :-------------------------- | :------------------------------------------------------------------------------------------------------------ | :---------------- | :------------------- | :---- | :-------------- | :------------- | :---------------- | :----- |
| Sher Mohammed & Co          | Gulab Shah Estate, Bus Depot, Gala No. 488, 2nd Lane, Saliya Galli, Lal Bahadur Shastri Marg, Kurla West, Kurla, Mumbai, Maharashtra 400070, India | +91 75060 07860 | None                 | None  | 5               | 58             | TMT Bars          | Mum

In [71]:
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner


# Define helper functions that might be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")



✅ Helper functions defined.


In [73]:

APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session

MODEL_NAME = "gemini-2.5-flash-lite"

# InMemorySessionService stores conversations in RAM (temporary)
session_service = InMemorySessionService()

# Step 3: Create the Runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")



# Run a conversation with two queries in the same session
# Notice: Both queries are part of the SAME session, so context is maintained
await run_session(
    runner,
    [
        "Hi,S M STEELS is a seller entity in Mumbai which sells TMT bars ",
        "Hello! What do you know about S M STEELS in Mumbai - what does it sell?",  # This time, the agent should remember!
    ],
    "stateful-agentic-session",
)



✅ Stateful agent initialized!
   - Application: default
   - User: default
   - Using: InMemorySessionService

 ### Session: stateful-agentic-session

User > Hi,S M STEELS is a seller entity in Mumbai which sells TMT bars 



User > Hello! What do you know about S M STEELS in Mumbai - what does it sell?
gemini-2.5-flash-lite >  S M STEELS is located at Avirahi Apts, B-501, SV Rd, Meghdoot, Hari Om Nagar, Borivali West, Mumbai, Maharashtra 400092, India. You can reach them at +91 78778 78282, and they have a Google rating of 4.5. S M STEELS sells TMT bars.
